In [ ]:
import requests
from collections import Counter
from bs4 import BeautifulSoup
import re
import csv
import nltk 
import numpy as np
from nltk import FreqDist

In [ ]:
# creates state list from csv file.   

with open("States_population.csv","r",encoding="utf-8") as file:
    csv_file = csv.reader(file)
    state_pop={}
    for row in csv_file:
        
        
        #clean and create dictionary with format {state,population}
        state = str(row[0]).replace("\xa0","").replace("\ufeff","")
        #remove commas from population
        population = str(row[1]).replace(",","")
        state_pop[state] = population
        

#create state list 
state_list = list(state_pop.keys())
state_list.pop()



In [ ]:
#This pulled all answers from the j-archive and created text file in form answer------year

#create list of all seasons (38 seasons total)
years = list(range(1,39))

#for each season, request the seasons main page and create list (cleaned_links) for each individual episode
for year in years:
    url = f"https://www.j-archive.com/showseason.php?season={year}"
    r = requests.get(url)   
    soup = BeautifulSoup(r.text, "xml")
    raw_links=[]
    cleaned_links = []
    
    #find all links
    for link in soup.find_all("a"):
        raw_links.append(link.get('href'))
        
    #filter to only include episode links. add to cleaned_links   
    for link in raw_links:
        if "https://www.j-archive.com/showgame" in link:
            cleaned_links.append(link)
            
    #for each episode (cleaned_links),  find all answers through regex        
    for link in cleaned_links:
        url=link
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "xml")
        s = str(soup.prettify)
        
        #regex 
        comp=re.compile("correct_response&quot;&gt;[^&]+")
        raw_answer=comp.findall(s)
        
        
        #clean answer and insert into text file with format (answer------year)
        for answer in raw_answer:
            full_answer = answer.replace("correct_response&quot;&gt;","")
            with open("j_archive_full_answers.txt","a",encoding='utf-8') as f:
                f.write(full_answer + "\t" + str(year))
                f.write("\n")
                
            
       

In [ ]:
# open text file of answers and create list with all answers
answer_list = []
with open("j_archive_full_answers.txt","r",encoding='utf-8') as j:
    for row in j:
        row = row.split("\t")
        answer_list.append(row[0])

        

In [ ]:
#creates frequency distributions input: answer_list output: frequency distributions
# full_list_no_exact(_fd) = answers with a state name in them but not the actual state
# full_list(_fd) = all answers with the state name in it
# exact_list(_fd) = Only exact matches to the actual state

full_list_no_exact = []
full_list = []
#frequency distribution of all state name mentions
for item in answer_list:
    for state in state_list:
        if state in item:
            full_list.append(item)
full_list_fd = FreqDist(full_list)
#frequency distribution of exact matches to state names           
exact_list = []
for a in full_list:
    if a in state_list:
        exact_list.append(a)
exact_list_fd = FreqDist(exact_list)

#frequency distribution of state names that aren't exact matches        
for item in full_list:
    if item not in exact_list:
        full_list_no_exact.append(item)
        
full_list_no_exact_fd = FreqDist(full_list_no_exact)        
        






In [ ]:
#clear contents of state frequency distribution text.
with open("state_frequency_distribution.txt","r+",encoding = "utf-8") as file:
    file.truncate(0)

In [ ]:
#state_pop[state] = population
#f.write(full_answer + "\t" + str(year))
#state_list
for state in exact_list_fd:
    answer_count = exact_list_fd[state]
    population = int(state_pop[state])
    #create ratio of answers per 1 million people
    answers_per = round(((answer_count/population) * 1000000),2)
    with open("state_frequency_distribution.txt","a",encoding = "utf-8") as state_file:
        state_file.write(state + "\t" + str(answer_count) + "\t" + str(population) + "\t" + str(answers_per)+ "\n")
        
        

In [ ]:
#included statistics
#total answers
print(len(answer_list))
#unique answers
print(len(set(answer_list)))


In [ ]:
# frequency distribution of full_answer_list
full_answer_fd = FreqDist(answer_list)

#50 most common Jeopardy answers
full_answer_fd.most_common(50)